To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
# from unsloth import FastLanguageModel
# import torch
# max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
#     "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
#     "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
#     "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
#     "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/gemma-2-9b-bnb-4bit",
#     "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
# ] # More models at https://huggingface.co/unsloth

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/Meta-Llama-3.1-8B",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
import json
from datasets import Dataset

# Load the emoji math dataset
file_path = "emoji_math_dataset.json"  # Adjust the path if needed
with open(file_path, "r") as f:
    data = json.load(f)

# Format for Unsloth: simple instruction-output pairs
formatted_data = []
for item in data:
    prompt = f"Solve this emoji math problem: {item['problem']}"
    solution_text = ", ".join([f"{key} = {value}" for key, value in item['solution'].items()])
    response = f"The correct answer is {solution_text}."
    formatted_data.append({"text": f"Instruction: {prompt}\nResponse: {response}"})

# Print the first example
print(formatted_data[0])


{'text': 'Instruction: Solve this emoji math problem: 🍎 + 🍎 = 10\nResponse: The correct answer is 🍎 = 5.'}


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 16,
#     lora_dropout = 0, # Supports any, but = 0 is optimized
#     bias = "none",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config = None, # And LoftQ
# )

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
import json


# Clear GPU memory
torch.cuda.empty_cache()

# Load LLaMA-3.1-8B with 4-bit quantization
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="meta-llama/Meta-Llama-3.1-8B-Instruct",  # Larger LLaMA model
    max_seq_length=256,  # Memory-efficient length
    dtype=torch.float16,
    load_in_4bit=True,
    device_map={"": torch.cuda.current_device()}
)

# Configure LoRA for fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj", "v_proj"],
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none"
)

# Training arguments
training_args = TrainingArguments(
    output_dir="emoji_math_solver_llama3",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=4,  # Adjust epochs as needed
    learning_rate=2e-5,
    fp16=True,
    logging_steps=5,
    save_steps=5,
    gradient_checkpointing=True,
    report_to="none",
)

# Convert formatted data into a Hugging Face dataset
from datasets import Dataset
dataset = Dataset.from_list(formatted_data)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=256,
    args=training_args
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("emoji_math_solver_llama3")
tokenizer.save_pretrained("emoji_math_solver_llama3")


==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Converting train dataset to ChatML (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 30 | Num Epochs = 4 | Total steps = 28
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 3,407,872/8,000,000,000 (0.04% trained)


Step,Training Loss
5,2.566700
10,2.604100
15,2.537500
20,2.454700
25,2.425600


('emoji_math_solver_llama3/tokenizer_config.json',
 'emoji_math_solver_llama3/special_tokens_map.json',
 'emoji_math_solver_llama3/tokenizer.json')

In [ ]:
!pip install pyngrok
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
import torch
import gradio as gr
from unsloth import FastLanguageModel

# # Load the fine-tuned emoji math solver model
# model_path = "emoji_math_solver_llama3"  # Adjust path if needed
# model, tokenizer = FastLanguageModel.from_pretrained(model_path)
# model = model.to("cuda")
# FastLanguageModel.for_inference(model)  # Enable inference mode

def test_model(model, tokenizer, test_cases):
    """Generate solutions for emoji math problems."""
    model.eval()
    results = []

    for test in test_cases:
        # Format input as per training prompt
        input_text = f"### Instruction: Solve this emoji math problem: {test}\n### Response: "
        inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

        # Set max_length to allow for response generation
        max_length = len(inputs["input_ids"][0]) + 700

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=max_length,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                temperature=0.5
            )

        # Decode and extract the solution part
        solution_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "### Response: " in solution_text:
            solution_text = solution_text.split("### Response: ")[-1].strip()

        results.append((test, solution_text))

    return results

def solve_emoji_math(input_cases):
    """Process user input and return emoji math solutions."""
    test_cases = [case.strip() for case in input_cases.split("\n") if case.strip()]
    results = test_model(model, tokenizer, test_cases)

    output = ""
    for problem, solution in results:
        output += f"Problem: {problem}\nSolution: {solution}\n\n"

    return output

# Create Gradio interface
iface = gr.Interface(
    fn=solve_emoji_math,
    inputs=gr.Textbox(
        lines=10,
        placeholder="Enter emoji math problems here, each on a new line (e.g., '🍎 + 🍎 + 🍎 = 15')",
    ),
    outputs="text",
    live=True,
    title="Emoji Math Solver",
    description="Enter emoji-based math problems, and get the correct answers from the fine-tuned LLaMA model."
)

# Launch the interface
iface.launch(share=True)  # Public URL in Colab

# Custom CSS for a clean look
custom_css = """
body {
    font-family: 'Arial', sans-serif;
    background-color: #e6f7ff; /* Light blue background */
}
.gradio-container {
    max-width: 800px;
    margin: 0 auto;
    padding: 20px;
}
h1 {
    color: #005f99; /* Dark blue title */
    text-align: center;
    font-size: 2.5em;
    margin-bottom: 20px;
}
textarea, .output-text {
    border: 2px solid #005f99 !important;
    border-radius: 10px;
    padding: 10px;
    font-size: 1.1em;
}
button {
    background-color: #005f99 !important; /* Blue button */
    color: white !important;
    border: none !important;
    border-radius: 10px;
    padding: 12px 20px;
    font-size: 1.2em;
    transition: background-color 0.3s;
}
button:hover {
    background-color: #004477 !important; /* Darker blue on hover */
}
.description {
    color: #333;
    text-align: center;
    font-size: 1.2em;
    margin-bottom: 20px;
}
.output-text {
    background-color: #fff;
    border: 2px solid #005f99;
    border-radius: 10px;
    padding: 15px;
    white-space: pre-wrap;
}
"""

# Add CSS styling
iface.css = custom_css


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2a30f6666accb841a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
